## Altcoin Trading

### Setup

In [15]:
import unittest, os, time
import json as json
from datetime import datetime
from pymarketcap import Pymarketcap
from bittrex.bittrex import Bittrex, API_V2_0, API_V1_1, BUY_ORDERBOOK, TICKINTERVAL_ONEMIN

# constants
IS_CI_ENV = True if 'IN_CI' in os.environ else False
TICKINTERVAL_ONEMIN = 'oneMin'
TICKINTERVAL_FIVEMIN = 'fiveMin'
TICKINTERVAL_HOUR = 'hour'
TICKINTERVAL_THIRTYMIN = 'thirtyMin'
TICKINTERVAL_DAY = 'Day'
STATS_HOUR = '1h'
STATS_DAY = '24h'
STATS_WEEK = '7d'

### Bittrex and CoinMarketCap Integration

In [14]:
# Need API keys for real-world Buy/Sell operations
#with open("secrets.json") as secrets_file:
#    secrets = json.load(secrets_file)
#    secrets_file.close()
#bittrex = Bittrex(secrets['key'], secrets['secret'], api_version=API_V1_1)

# without API keys (public functions)
bittrex = Bittrex(None,None,api_version=API_V1_1)
coinmarketcap = Pymarketcap()

### Useful Library Wrappers

In [ ]:
# INPUT:  ticker (string)
# OUTPUT: available balance (float)
def get_avail_balance(coin):
    return bittrex.get_balance(coin)['result']['Available']

# INPUT:  number_top_coins (int)
# OUTPUT: top N coin summaries
def get_all_markets(self):
    coins = bittrex.get_markets()['result']
    return [coins[i]['MarketName'] for i in range(len(coins))]

# INPUT:  market pair (string)
# OUTPUT: current ask price
def get_ask_price(market):
    return bittrex.get_ticker(market)['result']['Ask']

# INPUT:  coin symbol (string)
# OUTPUT: whether coin is available on Bittrex (Bool)
def is_on_bittrex(coin):
    markets = get_all_markets(None)
    name = 'BTC-'+coin
    c = bittrex.get_currencies()['result']
    currencies = [c[i]['Currency'] for i in range(len(c))]
    if coin in currencies and name in markets:
        return True
    return False

# INPUT:  num_coins (int)
# OUTPUT: list of coin symbols (string list)
def get_n_gainers(time, n):
    ret = []
    j = 0
    gainers = coinmarketcap.ranks('gainers',time)[time]
    for coin in gainers:
        if is_on_bittrex(coin['symbol']):
            ret.append(coin['symbol'])
            j += 1
        if j == n:
            break
    return (len(ret), list(ret))

# INPUT:  num_coins (int)
# OUTPUT: list of coin symbols (string list)
def get_n_losers(time, n):
    ret = []
    j = 0
    losers = coinmarketcap.ranks('losers',time)[time]
    for coin in losers:
        if is_on_bittrex(coin['symbol']):
            ret.append(coin['symbol'])
            j += 1
        if j == n:
            break
    return (len(ret), list(ret))

def sort_coins_by_vol(coins):
    #coins.sort(key=operator.itemgetter('24h_volume_usd'))
    return sorted(coins, key=lambda k: k['24h_volume_usd']) 

### Simulator

In [12]:
class Simulator:
    
    # initialize
    def __init__(self, tx_fee=0.01, start=1.0, verbose=False):
        self.start = start
        self.btc = start
        self.tx_fee = 0.01
        self.verbose = verbose
        self.holdings = 0.0
        self.fees_paid = 0.0 
        if verbose is True:
            print("Simulator initialized.")
    
    # get current balance (in BTC)
    def get_btc_balance(self):
        return self.btc
    
    # print current balance (in BTC)
    def print_balance(self):
        print("Simulator BTC Balance: %.5f (start = %.2f)" % (self.btc, self.start))   
    
    # print change between starting and current balances
    def print_change(self):
        x = ((-1 * self.fees_paid) + (-1 * self.holdings)) / self.start * 100.0
        print("Net change in value was %.3f%%" % x)
    
    # BUY order
    def buy(self, coin, amt_in_btc):
        start = time.time()
        # get market price
        price = get_ask_price('BTC-'+str(coin))
        if price is None:
            return 0
        # calculate how many coins we can buy (factoring in fee)
        num_to_buy = ((1.0 - self.tx_fee) * amt_in_btc) / price
        self.fees_paid += self.tx_fee * amt_in_btc
        self.btc -= amt_in_btc
        self.holdings += num_to_buy * price
        if self.verbose is True:
            print(str(datetime.now())+" - bought %.3f %s at %.7f BTC for a total purchase of %.7f BTC." % (num_to_buy, coin, price, amt_in_btc))
            #print("Buy execution time: %.2f s" % (time.time()-start))
        return num_to_buy       
    
    # SELL order
    def sell(self, coin, num):
        start = time.time()
        price = get_ask_price('BTC-'+str(coin))
        if price is None:
            return 0
        amt_profit = (1.0 - self.tx_fee) * num * price
        self.fees_paid += self.tx_fee * num * price
        self.btc += amt_profit
        self.holdings -= num * price
        if self.verbose is True:
            print(str(datetime.now())+" - sold %.3f %s at %.7f BTC for a total sale of %.7f BTC." % (num, coin, price, amt_profit))
            #print("Sell execution time: %.2f s" % (time.time()-start))
        return num     

### Strategy 1:  Top-N Flip

In [13]:
# buy and sell the top N coins by hourly market gain
class TopNTrader:
    
    # initialize
    def __init__(self, top_n=3, mins_to_wait=5, mins_to_run=60, verbose=False, strategy=STATS_HOUR):
        self.top_n = top_n
        self.verbose = verbose
        self.strategy = strategy
        self.mins_to_wait = mins_to_wait
        self.mins_to_run = mins_to_run
        self.num_bought = [0]*self.top_n
        self.num_sold = [0]*self.top_n
        self.balance = 0.0
        self.coins = []
        if verbose is True:
            print("TopNTrader initialized.")
    
    # get top N coins according to strategy (either losers or gainers)
    def get_top_n(self):
        (l, x) = get_n_losers(self.strategy, self.top_n)
        self.top_n = l
        return x
    
    # pause
    def wait(self):
        if self.verbose is True:
            print("Waiting for %.2f minutes..." % self.mins_to_wait)
        time.sleep(self.mins_to_wait * 60.0)        
    
    # BUY hook (invokes user-specified BUY function)
    def buy(self, f_buy):
        self.coins = self.get_top_n()
        for i in range(self.top_n):
            self.num_bought[i] = f_buy(self.coins[i], (self.balance/self.top_n))  
    
    # SELL hook (invokes user-specified SELL function)
    def sell(self, f_sell):
        for i in range(self.top_n):
            self.num_sold[i] = f_sell(self.coins[i], self.num_bought[i])
    
    # main trading loop
    def trade(self, f_buy, f_sell, sim):
        timeout = time.time() + 60*self.mins_to_run
        while time.time() < timeout:
            self.balance = sim.get_btc_balance()
            self.buy(f_buy)
            self.wait()
            self.sell(f_sell)
            if self.verbose is True:
                print("%.2f minutes remaining..." % ((timeout-time.time())/60))

### SIMULATE AUTOTRADING

In [ ]:
# simulator instance
sim = Simulator(tx_fee = 0.0007, verbose = True, start=1.0)    

# Top-N Trader instance
top_n = TopNTrader(top_n=10,mins_to_wait=0,mins_to_run=5,verbose=True, strategy=STATS_DAY)

# trade
top_n.trade(sim.buy, sim.sell, sim)

# results
sim.print_balance()
sim.print_change()